In [1]:
import pandas as pd
import numpy as np
import os
from src.data.process_dataset import load_csv_dataset, create_grid, create_grid_ids, correlation_adjacency_matrix, features_targets_and_externals, Dataset
from src.data.encode_externals import encode_times
from src.models.models import ExternalLSTM, GraphModel, CustomTemporalSignal, Encoder, Decoder, STGNNModel
from torch_geometric_temporal.signal import StaticGraphTemporalSignal, temporal_signal_split
import dill
from torch_geometric_temporal.nn.recurrent import DCRNN
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.data import Batch, DataLoader
from torch_geometric.nn import GCNConv

In [2]:
open_file = open("/Users/theisferre/Documents/SPECIALE/Thesis/data/processed/202106-citibike-tripdata.pkl", "rb")
dataset = dill.load(open_file)
train_dataset, test_dataset = Dataset.train_test_split(dataset, num_history=8)

In [3]:
num_nodes = dataset.num_nodes
num_features = dataset.num_features
num_weather_features = dataset.weather_information.shape[1]
num_time_features = dataset.time_encoding.shape[1]
node_hidden_features = 5
hidden_size = 16
graph_model = GraphModel(node_in_features=num_features, num_nodes=num_nodes, node_out_features=node_hidden_features)
weather_model = ExternalLSTM(num_weather_features, out_features=node_hidden_features, num_nodes=num_nodes, external_feat="weather", hidden_size=hidden_size)
time_model = ExternalLSTM(num_time_features, out_features=node_hidden_features, num_nodes=num_nodes, external_feat="time_encoding", hidden_size=hidden_size)

In [4]:
train_dataset[0]

CustomData(edge_attr=[684], edge_index=[2, 684], time_encoding=[8, 43], weather=[8, 4], x=[1, 8, 100], y=[100])

In [5]:
data_list = []
for i in range(len(train_dataset)):
    data_list.append(train_dataset[i])

In [6]:
b = Batch.from_data_list(data_list)
b

Batch(batch=[568], edge_attr=[388512], edge_index=[2, 388512], ptr=[569], time_encoding=[568, 8, 43], weather=[568, 8, 4], x=[568, 8, 100], y=[568, 100])

In [7]:
cell_state_graph, hidden_state_graph = graph_model(b)
cell_state_weather, hidden_state_weather = weather_model(b)
cell_state_time, hidden_state_weather = time_model(b)

In [8]:
print(cell_state_graph.shape)
print(cell_state_weather.shape)
print(cell_state_time.shape)

torch.Size([568, 500])
torch.Size([568, 500])
torch.Size([568, 500])


In [9]:
encoder = Encoder(
    node_in_features=num_features,
    num_nodes=num_nodes,
    node_out_features=node_hidden_features,
    time_features=num_time_features,
    weather_features=num_weather_features,
    hidden_size=hidden_size
)

decoder = Decoder(
    node_out_features=node_hidden_features,
    num_nodes=num_nodes
)

model = STGNNModel(encoder, decoder)

In [10]:
loader = DataLoader(data_list, batch_size=6)

In [11]:
batch.x[:, -1, :].shape

NameError: name 'batch' is not defined

In [14]:
for batch in loader:
    out, _ = model(batch)
    break

torch.Size([6, 1, 100])
torch.Size([6, 1, 100])
torch.Size([1, 6, 500])
torch.Size([1, 6, 500])


In [15]:
out.shape

torch.Size([6, 1, 100])

In [13]:
conv(loader_batch.x[:, 0, :].reshape(-1, 1), loader_batch.edge_index).shape

torch.Size([300, 7])

In [19]:
loader_batch.x

tensor([[ 1.2254,  1.5129,  1.3614,  ...,  0.0000,  0.0000,  0.0000],
        [-0.9518, -0.8223, -0.7880,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2936, -0.3123, -0.5731,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.9518, -0.9297, -0.2507,  ...,  0.0000,  0.0000,  0.0000],
        [-1.0531, -0.8894, -0.8955,  ...,  0.0000,  0.0000,  0.0000],
        [-1.0025, -1.0773, -0.7880,  ...,  0.0000,  0.0000,  0.0000]])

In [30]:
loader_batch.x.reshape(100, -1).shape

torch.Size([100, 24])

In [34]:
loader_batch.edge_index.shape

torch.Size([2, 2052])

In [36]:
conv(loader_batch.x.reshape(-1, 100*4), loader_batch.edge_index)

RuntimeError: index 17 is out of bounds for dimension 0 with size 6

In [56]:
for i in range(8):
    dat = train_dataset[0].x[i].unsqueeze(-1)
    dat_batch = dat.unsqueeze(0).repeat(3, 1, 1)
    print(dat_batch.shape)
    print(conv(dat_batch, train_dataset[0].edge_index).shape)

torch.Size([3, 100, 1])
torch.Size([3, 100, 7])
torch.Size([3, 100, 1])
torch.Size([3, 100, 7])
torch.Size([3, 100, 1])
torch.Size([3, 100, 7])
torch.Size([3, 100, 1])
torch.Size([3, 100, 7])
torch.Size([3, 100, 1])
torch.Size([3, 100, 7])
torch.Size([3, 100, 1])
torch.Size([3, 100, 7])
torch.Size([3, 100, 1])
torch.Size([3, 100, 7])
torch.Size([3, 100, 1])
torch.Size([3, 100, 7])


In [44]:
train_dataset[0].x

torch.Size([8, 100])